In [24]:
import pandas as pd
import numpy as np
from src.data.metrics import get_metrics

In [2]:
df_all = pd.read_json("../reports/all_models.jsonl", lines=True, orient="records")
metrics = ["top_1_accuracy", "top_5_accuracy", "macro_f1"]
get_metrics(df_all, metrics)
dfm = (
    df_all.groupby(["model_cls", "dataset", "loc_levels", "n_users"])[metrics]
    .agg(["mean", "std"])
    .reset_index()
)

In [9]:
dfm.loc[dfm["loc_levels"] == 2, "loc_levels"] = 1

In [10]:
dft = dfm[(dfm["n_users"].isin([150, 800]))]
dft = dft.drop(columns=["n_users"])
dft = dft.set_index(["loc_levels", "dataset", "model_cls"])
dft = dft.sort_index()
dft *= 100
dft

/tmp/ipykernel_3159265/1105105262.py:2: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  dft = dft.drop(columns=["n_users"])


top_1_accuracy           top_5_accuracy  \
                                              mean       std           mean   
loc_levels dataset        model_cls                                           
1          foursquare_NYC BiTULER        60.118266  0.044682      67.200823   
                          DeepTUL        58.716064  0.073530      65.484008   
                          MainTUL        55.666961  0.103056      62.610017   
                          T3S            52.984090  0.056556      60.282597   
                          TULERG         58.866673  0.061295      65.740468   
                          TULERL         58.469078  0.086665      65.279493   
                          TULHOR         53.853442  0.079035      61.130060   
                          TULVAE         59.786011  0.256308      66.768271   
           foursquare_TKY BiTULER        61.280729  0.058220      73.083263   
                          DeepTUL        59.143229  0.048352      70.660077   
                          MainTUL        56.807176  0.111303      69.178225   
                          T3S            53.651492  0.138275      66.303084   
                          TULERG         60.329396  0.126472      71.977088   
                          TULERL         59.680032  0.048159      71.310583   
                          TULHOR         54.386561  0.029319      67.238705   
                          TULVAE         54.189438  0.107752      64.922651   
           geolife        BiTULER        37.561985  0.103121      70.849244   
                          DeepTUL        36.322674  0.249306      72.643005   
                          MainTUL        33.998672  0.161429      70.259654   
                          T3S            35.251259  0.192028      71.107727   
                          TULERG         38.162508  0.258505      73.742533   
                          TULERL         35.257507  0.262005      68.790754   
                          TULHOR         34.652298  0.064479      72.459490   
                          TULVAE         37.082504  0.214314      70.454102   
4          foursquare_NYC BiTULER        61.605737  0.044905      70.781470   
                          DeepTUL        59.774903  0.065483      68.520043   
                          MainTUL        57.107713  0.074887      66.942076   
                          T3S            54.693391  0.044080      63.416642   
                          TULERG         60.532196  0.068301      69.756281   
                          TULERL         60.482211  0.106460      69.533797   
                          TULHOR         55.560456  0.038880      64.287955   
                          TULVAE         61.573720  0.112909      70.497566   
           foursquare_TKY BiTULER        62.723442  0.076480      75.951833   
                          DeepTUL        60.087991  0.087761      73.095262   
                          MainTUL        58.447869  0.079433      72.671304   
                          T3S            55.363035  0.067719      69.142229   
                          TULERG         61.865813  0.124423      74.999357   
                          TULERL         61.654120  0.117385      74.712243   
                          TULHOR         56.249518  0.147877      69.988144   
                          TULVAE         57.086006  0.111479      69.211936   
           geolife        BiTULER        46.277771  0.212815      77.825934   
                          DeepTUL        44.917418  0.198431      78.951232   
                          MainTUL        42.191246  0.202824      76.442935   
                          T3S            42.234977  0.490186      75.904104   
                          TULERG         47.369490  0.199141      79.777439   
                          TULERL         43.217368  0.396007      75.088829   
                          TULHOR         42.851119  0.276655      77.489360   
                          TULVAE         45.096248  0.114291      77.020811   

      

In [49]:
dfl = dft.xs("mean", axis=1, level=1, drop_level=True)
dfl = dfl.rename(
    columns={
        "top_1_accuracy": "Acc@1",
        "top_5_accuracy": "Acc@5",
        "macro_f1": "Macro F1",
    }
)
dfl = dfl.pivot_table(index=["loc_levels", "model_cls"], columns="dataset")
dfl.columns = dfl.columns.reorder_levels([1, 0])
dfl = dfl.sort_index(axis=1)
dfl = dfl[
    np.isin(
        dfl.index.get_level_values(1),
        ["BiTULER", "TULVAE", "DeepTUL", "MainTUL", "T3S", "TULHOR"],
    )
]
dfl = dfl.rename(
    columns={
        "foursquare_NYC": "Foursquare-NYC",
        "foursquare_TKY": "Foursquare-TKY",
        "geolife": "GeoLife",
    }
)
dfl.rename_axis(["Emb. Levels", "Model"], axis=0, inplace=True)

In [50]:
def highlight_max(s):
    """Highlight maximum value in bold for a Series."""
    is_max = s == s.max()
    return ["font-weight: bold" if v else "" for v in is_max]


df1 = dfl.xs(1, level=0)
df1 = df1.loc[["BiTULER", "TULVAE", "DeepTUL", "MainTUL", "T3S", "TULHOR"]]

latex_output = (
    df1.style.format(precision=2)
    .apply(highlight_max)
    .to_latex(hrules=True, convert_css=True)
)
print(latex_output)


\begin{tabular}{lrrrrrrrrr}
\toprule
dataset & \multicolumn{3}{r}{Foursquare-NYC} & \multicolumn{3}{r}{Foursquare-TKY} & \multicolumn{3}{r}{GeoLife} \\
 & Acc@1 & Acc@5 & Macro F1 & Acc@1 & Acc@5 & Macro F1 & Acc@1 & Acc@5 & Macro F1 \\
Model &  &  &  &  &  &  &  &  &  \\
\midrule
BiTULER & \bfseries 60.12 & \bfseries 67.20 & \bfseries 57.83 & \bfseries 61.28 & \bfseries 73.08 & \bfseries 58.96 & \bfseries 37.56 & 70.85 & 26.69 \\
TULVAE & 59.79 & 66.77 & 57.32 & 54.19 & 64.92 & 49.85 & 37.08 & 70.45 & 25.25 \\
DeepTUL & 58.72 & 65.48 & 56.60 & 59.14 & 70.66 & 56.90 & 36.32 & \bfseries 72.64 & \bfseries 29.82 \\
MainTUL & 55.67 & 62.61 & 53.01 & 56.81 & 69.18 & 54.09 & 34.00 & 70.26 & 21.76 \\
T3S & 52.98 & 60.28 & 49.50 & 53.65 & 66.30 & 50.38 & 35.25 & 71.11 & 21.52 \\
TULHOR & 53.85 & 61.13 & 50.40 & 54.39 & 67.24 & 51.06 & 34.65 & 72.46 & 24.92 \\
\bottomrule
\end{tabular}



In [77]:
df2 = dfl.xs(4, level=0)
df2 = df2.loc[["BiTULER", "TULVAE", "DeepTUL", "MainTUL", "T3S", "TULHOR"]]

# Calculate differences between df2 and df1
df_diff = df2 - df1

# Color entries in df_diff based on value
style = df_diff.style.background_gradient(
    cmap="RdYlGn", vmin=-12, vmax=12, axis=None, text_color_threshold=0.2
).format("{:+.2f}")
style


In [78]:
print(style.to_latex(hrules=True, convert_css=True))

\begin{tabular}{lrrrrrrrrr}
\toprule
dataset & \multicolumn{3}{r}{Foursquare-NYC} & \multicolumn{3}{r}{Foursquare-TKY} & \multicolumn{3}{r}{GeoLife} \\
 & Acc@1 & Acc@5 & Macro F1 & Acc@1 & Acc@5 & Macro F1 & Acc@1 & Acc@5 & Macro F1 \\
Model &  &  &  &  &  &  &  &  &  \\
\midrule
BiTULER & {\cellcolor[HTML]{E8F59F}} \color[HTML]{000000} +1.49 & {\cellcolor[HTML]{BFE47A}} \color[HTML]{000000} +3.58 & {\cellcolor[HTML]{E3F399}} \color[HTML]{000000} +1.71 & {\cellcolor[HTML]{E8F59F}} \color[HTML]{000000} +1.44 & {\cellcolor[HTML]{CFEB85}} \color[HTML]{000000} +2.87 & {\cellcolor[HTML]{E8F59F}} \color[HTML]{000000} +1.44 & {\cellcolor[HTML]{36A657}} \color[HTML]{000000} +8.72 & {\cellcolor[HTML]{6BBF64}} \color[HTML]{000000} +6.98 & {\cellcolor[HTML]{7FC866}} \color[HTML]{000000} +6.26 \\
TULVAE & {\cellcolor[HTML]{E2F397}} \color[HTML]{000000} +1.79 & {\cellcolor[HTML]{BDE379}} \color[HTML]{000000} +3.73 & {\cellcolor[HTML]{DFF293}} \color[HTML]{000000} +2.04 & {\cellcolor[HTML]{CFEB85}}